In [4]:
search_for = 124
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002232789993286133
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 621252555
type: [1, 1, 1, 1, 124] 124
cases of this type: 236421376
100000 21.326019512928976
200000 38.7041449018354
300000 46.09422977618929
400000 46.332857500464314
500000 47.78689866330247
600000 46.84646364995623
700000 45.39255612895956
800000 46.03707069096337
900000 45.51685634333872
1000000 15.442242516081729
1100000 46.46427395293117
1200000 45.213811332230335
1300000 45.328179924952764
1400000 45.56192067864545
1500000 47.40959073436359
1600000 45.61884376322019
1700000 45.38772881165088
1800000 46.343004120593754
1900000 39.17338949990483
2000000 22.272505104802722
2100000 43.711157782905865
2200000 43.7266449234304
2300000 42.64238716398929
2400000 44.052354807623324
2500000 43.434769045604234
2600000 42.331598763425106
2700000 42.00501036814794
2800000 42.5932456766007
2900000 17.713270916904392
3000000 40.6771904

29700000 41.915107573884825
29800000 39.928643652768976
29900000 40.82760727633572
30000000 42.67291276196907
30100000 43.68570689953129
30200000 43.63483049426227
30300000 26.726125402849316
30400000 38.625902005862606
30500000 45.43942363743787
30600000 46.92157059384027
30700000 46.04166569147108
30800000 20.076567830293744
30900000 46.29024032425413
31000000 46.15906630059929
31100000 45.77850926731901
31200000 44.69435088801685
31300000 41.245111831917555
31400000 40.91013867943232
31500000 17.119773307329147
31600000 41.22828489570011
31700000 40.62717156003219
31800000 43.04916542961667
31900000 43.93134518517483
32000000 45.63302766959784
32100000 46.233983864644465
32200000 24.902827352406312
32300000 42.11231536901658
32400000 45.71178069374245
32500000 45.15572151677218
32600000 44.52507045549868
32700000 26.782346624836833
32800000 38.59015219124437
32900000 45.218293761219385
33000000 42.31188536179942
33100000 42.49285197095994
33200000 40.15262267286234
33300000 39.11062

59800000 37.51995964255098
59900000 37.14654646321825
60000000 38.21335984424478
60100000 15.197334648764503
60200000 38.64346390019568
60300000 37.10031163714604
60400000 37.91040375423855
60500000 37.77178173663557
60600000 20.06434566910977
60700000 39.98400676175454
60800000 41.045048252366215
60900000 40.7117496510725
61000000 44.40291870300641
61100000 43.88865025556786
61200000 42.74896294447395
61300000 42.707819157844895
61400000 42.168463430860825
61500000 33.932977242282874
61600000 25.086494477341105
61700000 38.41642722596279
61800000 40.31337243063882
61900000 38.22759362687056
62000000 18.191259186601584
62100000 36.054417486411616
62200000 40.030708413949625
62300000 37.28969128563646
62400000 37.23861150473715
62500000 21.830273000008013
62600000 41.383917758484614
62700000 41.302135261081425
62800000 42.62921744585107
62900000 43.23696827609072
63000000 42.69479339108366
63100000 41.718213405063146
63200000 41.5138500539762
63300000 39.57924374576599
63400000 39.33607

89900000 35.87758109422582
90000000 34.92853270549766
90100000 35.39502599856815
90200000 35.07148060214768
90300000 34.17325771063506
90400000 16.582250170268868
90500000 32.413568108233086
90600000 16.055179364963323
90700000 31.889859531448522
90800000 26.569147888969457
90900000 25.490827661675322
91000000 35.063012806388855
91100000 34.36548024395677
91200000 34.68680581904195
91300000 34.99097600418474
91400000 36.959377146936006
91500000 37.95568037005752
91600000 39.487741297654814
91700000 39.01298290132903
91800000 37.991945585085105
91900000 35.04825968432873
92000000 36.28069303241664
92100000 35.365111362707545
92200000 35.29360639441021
92300000 16.204586550537872
92400000 35.36279318018747
92500000 17.109722559409988
92600000 31.607493100161157
92700000 23.77897446207989
92800000 28.520140871437867
92900000 35.028199128143015
93000000 35.61087027600423
93100000 35.46809474971232
93200000 35.61856582349599
93300000 38.04709514338666
93400000 39.66833890090587
93500000 39.

119100000 3.8335094185058773
119200000 3.68667074989748
119300000 3.7517585757174396
119400000 3.683916859654516
119500000 3.714766684795075
119600000 3.680948375664496
119700000 3.7125555370705228
119800000 3.740387785759578
119900000 3.7373139940549334
120000000 3.685163211049706
120100000 3.6815823076344434
120200000 3.6773893106634916
120300000 3.7134945389648086
120400000 3.8175941581274535
120500000 3.820050128039744
120600000 3.782875343940725
120700000 3.660315991336776
120800000 3.829807938317173
120900000 3.763255040460368
121000000 3.8124771729900204
121100000 3.7595672785426175
121200000 3.7622051369999348
121300000 3.7509222959273094
121400000 3.7477673710448443
121500000 3.650273515203148
121600000 3.6594008129446207
121700000 3.7221841917487253
121800000 3.6763482585634093
121900000 3.686017252869579
122000000 3.7067070033369096
122100000 3.6847253955248
122200000 3.8079900473218724
122300000 3.7042942708394864
122400000 3.6654124759269955
122500000 3.7194151556801827
12

...

492700000 2.6770042751693097
492800000 3.1851913651239476
492900000 3.739583487198621
493000000 3.7839545255431997
493100000 4.0741332127482
493200000 3.8835766484904686
493300000 3.575535249786476
493400000 1.3740408385194605
493500000 3.36116305784749
493600000 3.058744033038093
493700000 2.8646026057748264
493800000 3.4333240156608227
493900000 3.7007291429611944
494000000 3.6993354990435012
494100000 4.105603882867665
494200000 3.5756616726994648
494300000 2.4542909696815673
494400000 2.523157860594541
494500000 3.286410942527387
494600000 2.4236521900382537
494700000 3.4815341160278317
494800000 3.7072238979420695
494900000 3.7355505960615676
495000000 3.5984831043156684
495100000 3.9090643833298353
495200000 3.699203372224556
495300000 1.298283249672721
495400000 3.4065119098005727
495500000 2.2980243377053142
495600000 3.459481540785044
495700000 3.598515585344328
495800000 3.609792822307739
495900000 3.424717418704927
496000000 3.540884365476721
496100000 3.878965032722695
4962

520600000 3.176250584088342
520700000 3.1514995094723335
520800000 3.058442112704605
520900000 3.0268841406334865
521000000 3.1733554970088735
521100000 3.0298704476508003
521200000 2.926726735658748
521300000 2.9013346369226474
521400000 3.006130545561006
521500000 3.04816901164188
521600000 3.056097010589305
521700000 2.995448319166809
521800000 3.0969347422076763
521900000 3.0808681746695865
522000000 3.219280410863598
522100000 3.141274670407414
522200000 3.1661373913416337
522300000 3.146553812274486
522400000 3.244813293118699
522500000 3.1829610680750076
522600000 3.141673790872902
522700000 3.1401797331825034
522800000 3.2522476791164214
522900000 3.1897927156928776
523000000 3.1895176650900745
523100000 3.062795272601489
523200000 3.2358617538221477
523300000 3.151918088036494
523400000 3.1858336590870056
523500000 2.8606822103096543
523600000 2.7716688497686683
523700000 2.819515648234052
523800000 2.9892230698229274
523900000 3.021107605430914
524000000 3.0593483657175966
52

547400000 2.899763602869378
547500000 2.1116680928078098
547600000 1.9723737661954825
547700000 2.8924208222390413
547800000 3.298665936049998
547900000 3.1153388910069504
548000000 1.752983216879209
548100000 2.778691397162994
548200000 3.099813299307135
548300000 1.0467937227912845
548400000 0.5622054428825677
548500000 0.5548811650777559
548600000 0.5652771199483771
548700000 0.5638885936469435
548800000 0.5638358815031151
548900000 0.5647243716637955
549000000 0.5630217658236623
549100000 0.5666317448352575
549200000 0.5601608061290615
549300000 0.557723261855334
549400000 0.5551586138076948
549500000 0.5537174213156502
549600000 0.5459144205877184
549700000 0.5443791200082798
549800000 0.5480909636584455
549900000 0.552057703730007
550000000 0.5500316585684253
550100000 0.5476212164082527
550200000 0.5505682633001804
550300000 0.5516986929704514
550400000 0.5532392152607574
550500000 0.5471454384726584
550600000 0.5667579790436724
550700000 3.4559514766425
550800000 3.284704024754

575800000 1.2351670403460395
575900000 1.3548914057404167
576000000 0.9329345362490414
576100000 0.9293056541057395
576200000 0.9148485399701661
576300000 1.2954683936401705
576400000 1.2312233648098507
576500000 1.0179748714856043
576600000 0.71986619805184
576700000 1.1260259917042057
576800000 1.296386685738951
576900000 1.0223796002090177
577000000 1.197661592350761
577100000 0.585155714181314
577200000 1.3036989936109284
577300000 1.3060727423749137
577400000 0.9429273128275275
577500000 1.1485978778197465
577600000 0.6740343806731238
577700000 1.3021196553245882
577800000 1.129659883069704
577900000 1.048471200694432
578000000 0.9428979135085676
578100000 0.8509439795326491
578200000 1.297763646171242
578300000 0.9341893597768447
578400000 1.2401853009010255
578500000 0.7466198323539098
578600000 1.0947422463462186
578700000 1.2912459030255081
best so far: 0
type: [1, 2, 1, 31, 2] 124
cases of this type: 953312
578800000 0.4153237096344233
578900000 0.27368554035005305
579000000 

599300000 0.1639904080602659
599400000 0.16401751344927154
599500000 0.16256767455950047
599600000 0.1627907215307984
599700000 0.16018206814285119
599800000 0.41448852145318854
599900000 0.793667459753073
600000000 0.9175970630506873
600100000 0.9120350921093987
600200000 0.5861973492842582
600300000 0.8249281299360792
600400000 0.5136335804018709
600500000 0.8043150280724666
600600000 0.7442820030627647
600700000 0.6235799266960158
600800000 0.8223554498712288
600900000 0.528305536081314
601000000 0.8151646418241759
601100000 0.6189058072623247
601200000 0.718731208507965
601300000 0.7010271056522462
601400000 0.5754026074737376
601500000 0.7855632300610145
601600000 0.5580436006768511
601700000 0.7516411662145092
601800000 0.6326253822090625
601900000 0.6298831231673062
602000000 0.7301261873672339
602100000 0.5902356792427104
602200000 0.8071280040571888
602300000 0.5583235367364288
602400000 0.6946252591940661
602500000 0.7084566142362986
602600000 0.5732620283171965
602700000 0.7